In [1]:
import os
import wandb

import numpy as np
import tensorflow as tf
import tensorflow.keras as tfk

from sklearn import metrics

2023-08-10 15:12:05.981380: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-08-10 15:12:05.981432: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
def mae(y_trues, y_preds):
    if not isinstance(y_trues, np.ndarray):
        y_trues = np.asarray(y_trues)

    if not isinstance(y_preds, np.ndarray):
        y_preds = np.asarray(y_preds)

    return np.mean(np.abs(y_trues-y_preds))

def rmse(y_trues, y_preds):
    if not isinstance(y_trues, np.ndarray):
        y_trues = np.asarray(y_trues)

    if not isinstance(y_preds, np.ndarray):
        y_preds = np.asarray(y_preds)

    return np.sqrt(np.mean(np.power(y_trues-y_preds, 2)))


def mrae(y_trues, y_preds):
    if not isinstance(y_trues, np.ndarray):
        y_trues = np.asarray(y_trues)

    if not isinstance(y_preds, np.ndarray):
        y_preds = np.asarray(y_preds)
    return np.mean(np.abs(np.divide(y_trues -y_preds, y_trues)))

In [3]:
x = np.array([[1, 2, 3], 
              [4, 5, 6]])
x

array([[1, 2, 3],
       [4, 5, 6]])

In [4]:
y = np.array([1, 2])
y

array([1, 2])

In [8]:
n_units = n_features = x.shape[1]
learning_rate = 1e-2
batch_size = 2
n_epochs = 100

n_units

3

In [5]:
class MLP (tf.keras.Model):
    def __init__(self, n_units, n_features, name="test", **kwargs):
        super(MLP, self).__init__(name=name, **kwargs)
        self.n_units = n_units
        self.n_features = n_features
        
        self.dense_1 = tf.keras.layers.Dense(units=self.n_units,
                                             input_shape=(self.n_features,))
        self.dense_2 = tf.keras.layers.Dense(units=self.n_units, )
        
    def call (self, inputs):
        x = self.dense_1(inputs)
#         x = self.dense_2(x)
        return x

In [6]:
losses_name = [tfk.losses.MeanAbsoluteError(), 
               tfk.losses.MeanSquaredError(), 
               tfk.losses.MeanAbsolutePercentageError(),
               tfk.losses.KLDivergence(),
              ]


experiments_name = ["MAE", "MSE", "MRAE", "KLD"]



In [10]:


for l in range(len(losses_name)):
    loss = losses_name[l]
    experiment_name = experiments_name[l]
    

    # 1. Start a W&B run
    
    run = wandb.init(name="MLP method", 
                     project='mlp-method', 
                     notes = 'Vanilla MLP method: to figure out which loss function works better',
                     entity='sorooshi',
                     group=experiment_name
                    )

#     mlp = tfk.Sequential([
#         tf.keras.layers.Dense(units=n_units,
#                               input_shape=(n_features,))
#     ])
    
    mlp = MLP(n_units=n_units, n_features=(n_features,))
    
    mlp.compile(optimizer=tfk.optimizers.Adam(learning_rate=learning_rate),
             loss=loss)

    
    # 2. Save model inputs and hyperparameters
    config = run.config
    config.learning_rate = learning_rate
    config.loss = experiment_name
    config.n_units = n_units

    callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=5)

    history = mlp.fit(x=x, y=x, epochs=n_epochs, 
                      batch_size=batch_size, 
                      # callbacks=[callback],
                      verbose=True)
    
    run.log({
        "mae" : mae(x, mlp.predict(x)),
        "mrae" : mrae(x, mlp.predict(x)),
        "rmse" : rmse(x, mlp.predict(x)),
    })
        
    # Save the model
    # model.save('path_to_my_model',save_format='tf')


#     mlp.save(os.path.join(wandb.run.dir,
#                           "MLP" + experiment_name + ".h5" ))

    # Recreate the exact same model purely from the file
    # new_model = keras.models.load_model('path_to_my_model')

    run.finish()


mae,▁
mrae,▁
rmse,▁
mae,0.32918
mrae,0.21306
rmse,0.49812


wandb: wandb version 0.15.8 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
2023-08-10 15:15:28.295943: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory


Epoch 1/100
1/1 [==============================] - 0s 94ms/step - loss: 4.2018
Epoch 2/100
1/1 [==============================] - 0s 964us/step - loss: 4.0868
Epoch 3/100
1/1 [==============================] - 0s 879us/step - loss: 3.9718
Epoch 4/100
1/1 [==============================] - 0s 1ms/step - loss: 3.8568
Epoch 5/100
1/1 [==============================] - 0s 1ms/step - loss: 3.7418
Epoch 6/100
1/1 [==============================] - 0s 866us/step - loss: 3.6268
Epoch 7/100
1/1 [==============================] - 0s 920us/step - loss: 3.5118
Epoch 8/100
1/1 [==============================] - 0s 842us/step - loss: 3.3968
Epoch 9/100
1/1 [==============================] - 0s 796us/step - loss: 3.2818
Epoch 10/100
1/1 [==============================] - 0s 985us/step - loss: 3.1902
Epoch 11/100
1/1 [==============================] - 0s 1ms/step - loss: 3.1257
Epoch 12/100
1/1 [==============================] - 0s 1ms/step - loss: 3.0587
Epoch 13/100
1/1 [============================

mae,▁
mrae,▁
rmse,▁
mae,0.15596
mrae,0.10656
rmse,0.23052


wandb: wandb version 0.15.8 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
2023-08-10 15:15:35.666904: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory


...

Epoch 1/100
1/1 [==============================] - 0s 132ms/step - loss: 129.4614
Epoch 2/100
1/1 [==============================] - 0s 2ms/step - loss: 125.6781
Epoch 3/100
1/1 [==============================] - 0s 2ms/step - loss: 121.8947
Epoch 4/100
1/1 [==============================] - 0s 2ms/step - loss: 118.1114
Epoch 5/100
1/1 [==============================] - 0s 830us/step - loss: 114.3281
Epoch 6/100
1/1 [==============================] - 0s 1ms/step - loss: 110.5447
Epoch 7/100
1/1 [==============================] - 0s 2ms/step - loss: 106.7614
Epoch 8/100
1/1 [==============================] - 0s 924us/step - loss: 102.9781
Epoch 9/100
1/1 [==============================] - 0s 772us/step - loss: 99.1947
Epoch 10/100
1/1 [==============================] - 0s 732us/step - loss: 95.4114
Epoch 11/100
1/1 [==============================] - 0s 697us/step - loss: 91.6281
Epoch 12/100
1/1 [==============================] - 0s 683us/step - loss: 89.3570
Epoch 13/100
1/1 [=========

mae,▁
mrae,▁
rmse,▁
mae,0.37488
mrae,0.10673
rmse,0.67304


wandb: wandb version 0.15.8 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
2023-08-10 15:15:50.413231: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory


Epoch 1/100
1/1 [==============================] - 0s 373ms/step - loss: 16.8899
Epoch 2/100
1/1 [==============================] - 0s 1ms/step - loss: 16.7254
Epoch 3/100
1/1 [==============================] - 0s 952us/step - loss: 16.5980
Epoch 4/100
1/1 [==============================] - 0s 759us/step - loss: 16.5165
Epoch 5/100
1/1 [==============================] - 0s 868us/step - loss: 16.4479
Epoch 6/100
1/1 [==============================] - 0s 1ms/step - loss: 16.3887
Epoch 7/100
1/1 [==============================] - 0s 2ms/step - loss: 16.3370
Epoch 8/100
1/1 [==============================] - 0s 1ms/step - loss: 16.2912
Epoch 9/100
1/1 [==============================] - 0s 867us/step - loss: 16.2502
Epoch 10/100
1/1 [==============================] - 0s 769us/step - loss: 16.2132
Epoch 11/100
1/1 [==============================] - 0s 694us/step - loss: 9.1506
Epoch 12/100
1/1 [==============================] - 0s 827us/step - loss: 8.8061
Epoch 13/100
1/1 [=================

mae,▁
mrae,▁
rmse,▁
mae,1.95106
mrae,0.6309
rmse,2.19476


In [11]:
print("finish")

finish


In [13]:
# mlp.loss(y_true=x, y_pred=x)

In [19]:
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor

from joblib import dump, load

In [28]:


run = wandb.init(name="MLP method", 
                     project='mlp-method', 
                     notes = 'Vanilla MLP method: to figure out which loss function works better',
                     entity='sorooshi',
                     group='RF'
                    )

In [60]:
rf_reg = RandomForestRegressor(n_estimators=100, 
                               n_jobs=-2, 
                               criterion='mse', 
                               min_samples_leaf=1,
                               verbose=1)
    
rf_reg.fit(x, x)

x_preds_rf = rf_reg.predict(x)
    
    

[Parallel(n_jobs=-2)]: Using backend ThreadingBackend with 15 concurrent workers.
[Parallel(n_jobs=-2)]: Done  20 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-2)]: Done 100 out of 100 | elapsed:    0.1s finished
[Parallel(n_jobs=15)]: Using backend ThreadingBackend with 15 concurrent workers.
[Parallel(n_jobs=15)]: Done  20 tasks      | elapsed:    0.0s
[Parallel(n_jobs=15)]: Done 100 out of 100 | elapsed:    0.0s finished


In [30]:
# filename = "rf_reg-" + name +  "-.joblib"
dump(rf_reg, os.path.join(wandb.run.dir,
                          "RF-" + "experiment_name" + ".joblib" ) )

run.log({
    "mae" : mae(x, mlp.predict(x)),
    "mrae" : mrae(x, mlp.predict(x)),
    "rmse" : rmse(x, mlp.predict(x)),
})
        
run.finish()

mae,6.54271
mrae,2.13689
rmse,8.11196
_runtime,7
_timestamp,1618265665
_step,0


mae,▁
mrae,▁
rmse,▁
_runtime,▁
_timestamp,▁
_step,▁


1. MAE  : 2.6526409313082695

2. MSE  : 1.0456066777308781

3. MRAE : 5.888582201053699

4. COSS : 3.802344910800457

5. KLD  : 2.9339942882458367

In [83]:
def init_a_wandb(name, project, notes, group):
    
    """ name := the within the project name, e.g., RF-reg-1
        project := the project name, e.g., Non-sequential Regressions 
        notes := Description, e.g., Non-sequential Regressions Comparison for SuperOX
        group := name of experiment or the algorithm under consideration, e.g., RF-1
        
    
    """
    
    run = wandb.init(name=name, 
                     project=project, 
                     notes = notes,
                     entity='sorooshi',
                     group=group
                    )
        
    return run



In [84]:
def wandb_metrics(run, y_preds, y_trues):
    
    run.log({
        "MAE" : mae(y_trues=y_trues, y_preds=y_preds),
        "MRAE" : mrae(y_trues=y_trues, y_preds=y_preds),
        "RMSE" : rmse(y_trues=y_trues, y_preds=y_preds),
        "R^2-Score": metrics.r2_score(y_trues, y_preds)
    })
    
    
    return run



In [85]:
def save_model(run, model, name, experiment_name):
    if name == "rf" or name == "gbr":
        dump(model, os.path.join(wandb.run.dir,
                          name + experiment_name + ".joblib"))
    else:
        model.save(os.path.join(wandb.run.dir,
                                name + experiment_name + ".h5"))
    return run


In [90]:
A = init_a_wandb(name=None, 
                 project="Non-sequential Regressions", 
                 notes="Non-sequential Regressions Comparison for SuperOX", 
                 group="RF")



MAE,0.645
MRAE,0.2725
RMSE,0.64657
R^2-Score,0.8142
_runtime,2
_timestamp,1618312988
_step,0


MAE,▁
MRAE,▁
RMSE,▁
R^2-Score,▁
_runtime,▁
_timestamp,▁
_step,▁


In [91]:
wandb_metrics(run=A, y_trues=x, y_preds=x_preds_rf)

In [5]:
tmp = 0
from tqdm import tqdm
for i in tqdm(range(10000000)):
    tmp += i

100%|██████████| 10000000/10000000 [00:03<00:00, 2890479.31it/s]
